[View in Colaboratory](https://colab.research.google.com/github/seraffin/FailOmen/blob/master/Mariusz/workingNN.ipynb)

In [0]:
import tensorflow as tf

# Helper libraries
import matplotlib.pyplot as plt
import numpy as np

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
!ls

gdrive	sample_data


In [0]:
import pandas as pd

#Importing dataset
dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/convertDataToML.csv', index_col=False)

In [0]:
x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

In [8]:
y.shape

(4813, 542)

In [9]:
failCount = 0
passCount = 0
failBuildCount = 0

for a in y:
  if a[np.argmin(a)] == 0 : failBuildCount += 1
  for b in a:
    if b == 0 : failCount += 1
    if b == 1 : passCount += 1
print (failBuildCount)
print (failCount)
print (passCount)
print (failCount / passCount * 100, '%', sep='')

632
973
2607673
0.037312960635785236%


In [0]:
#from sklearn.preprocessing import LabelEncoder
#labelencoder = LabelEncoder()
#y = labelencoder.fit_transform(y)

In [0]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2)

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense
#12140
model = Sequential()
# Adding the input layer and the first hidden layer
model.add(Dense(6070, activation="sigmoid", input_dim=23739, kernel_initializer="uniform")) # TRY smaller input_dim value or less neurons
# Adding the second hidden layer
#model.add(Dense(12140, activation = "sigmoid", kernel_initializer="uniform"))
# Adding the output layer
model.add(Dense(542, activation="sigmoid", kernel_initializer="uniform"))

Using TensorFlow backend.


In [13]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(xTrain, yTrain, batch_size = 20, epochs = 1)

Epoch 1/1
3850/3850 [==============================] - 40s 10ms/step - loss: 0.0107 - acc: 0.9969


In [14]:
lossTest, accTest = model.evaluate(xTest, yTest)

print('Test accuracy:', accTest)
print('Test loss:', lossTest)


963/963 [==============================] - 1s 555us/step
Test accuracy: 0.9995286967276537
Test loss: 0.007232388066737958


In [0]:
predictions = model.predict(xTest)

In [41]:
#Distributions of argmins through all the predictions
i = 1
tab = [0] * 542
for a in predictions:
  j = 0
  for b in a:
    if b < 1.0 : j = j + 1
    
  #print (i, '. ', j, np.argmin(a))
  tab[np.argmin(a)] += 1
  i = i + 1

i = 0
for a in tab:
  if a > 0 : print ('position', i, '\targmin count', a)
  i += 1

position 0 	argmin count 957
position 221 	argmin count 1
position 247 	argmin count 1
position 317 	argmin count 1
position 422 	argmin count 2
position 513 	argmin count 1


In [0]:
#Multiple fails in one build on test set
i = 1
for a in yTest:
  j = 0
  for b in a:
    if b < 1.0 : j += 1
  
  if j > 2 : print (i, '. ', j)
  i = i + 1

In [39]:
lenght = len(yTest)

failPositions = [[] for y in range(lenght)]

for i, a in enumerate(yTest):
  
  for j, b in enumerate(a):
    if b == 0 : failPositions[i].append(j);
      

additionalPredictions = 3
predictionsTemp = predictions
predictionPositions = [[] for y in range(lenght)]

for i, a in enumerate(predictionsTemp):
  
  for j in range(len(failPositions[i]) + additionalPredictions):
    argmin = np.argmin(a)
    #print(np.argmin(a), a[argmin])
    predictionPositions[i].append(argmin)
    predictionsTemp[i][argmin] = 1

print (predictionPositions)
predictionHits = [[] for y in range(lenght)]

for i, a in enumerate(failPositions):
  
  for j, b in enumerate(a):
    count = 0
    
    for c in predictionPositions[i]:
      #print(predictions[i][c])
      predictionHits[i].append(predictions[i][c])
      if c == b : count += 1
        
    [count] + predictionHits[i]

#print (predictionHits)

[[0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [422, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [422, 0, 0], [422, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [221, 247, 413], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0], [332, 413, 422], [0, 0, 0], [422, 0, 0], [0, 0, 0], [0, 0, 0], [247, 317, 413], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [

In [40]:
print(np.argmin(predictions[0]))
print(predictions[0][319])
print(predictions[0])

0
1.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [0]:
i = 1
for a in predictions:
  argmin = np.argmin(a)
  failPrediction = yTest[i][argmin]
  argminY = np.argmin(yTest[i])
  fail = yTest[i][argminY]
  print(i, '. ', a[argmin], ' => ', fail, 'NOPE' if argmin != argminY else "JUP")
  i = i + 1